Loading Data

In [1]:
import pandas as pd
import numpy as np
import os

# Obtaining current working directory (necessary for when running with another folder as "Project" in VSCode)
current_directory = os.getcwd()
file_name = f'{current_directory}/../datasets/cigarettes_treated.feather'

# -------------
# Loading Data
# -------------
sls = pd.read_feather(file_name)
sls['Date'] = pd.to_datetime(sls['Date'])

Generating Stocks

In [2]:
# ----------------------------------------------------
# Generating a random weekday to re-stock a product
# ----------------------------------------------------
stocks = sls.groupby(by=['Store_ID2', 'Product_ID'], as_index=False).agg({'Date': ['min', 'max']})
stocks.columns = [f'{a}_{b}' if len(b) > 0 else a for (a,b) in stocks.columns ]
stocks['Date_min'] = stocks['Date_min'] - pd.Timedelta(days=1)
stocks['Date'] = stocks.apply(lambda row: pd.date_range(row['Date_min'], row['Date_max'], freq='7D'), axis=1)


stocks = stocks[['Store_ID2', 'Product_ID', 'Date']].explode('Date')

In [7]:
sls.groupby(by=['Store_ID2', 'Product_ID', 'Date'], as_index=False)['Quantity'].transform(lambda x: x.rolling(window=7).sum())

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
215444     NaN
215445    26.0
215446    64.0
215447     7.0
215448    83.0
Name: Quantity, Length: 214888, dtype: float64

In [19]:
# Adding sales of next 7 days
stocks['Next_restock_date'] = stocks['Date'] + pd.Timedelta(days=7)

# Agregating sales with granularity wanted
tmp = sls.groupby(by=['Store_ID2', 'Product_ID', 'Date'], as_index=False)['Quantity'].sum()

,Store_ID2,Product_ID,Date,Quantity,Sales_Previous_7_days
0,0,0,2016-01-18,27.0,NaN
1,0,9,2016-01-18,21.0,NaN
2,0,10,2016-01-18,1.0,NaN
3,0,44,2016-01-18,2.0,NaN
4,1,0,2016-01-18,60.0,NaN
...,...,...,...,...,...
91174,3034,533,2022-07-04,10.0,NaN
91175,3034,2341,2022-07-04,20.0,NaN
91176,3035,3,2022-07-09,20.0,NaN
91177,3035,100,2022-07-09,20.0,NaN


In [48]:
tmp = tmp.sort_values(by=['Store_ID2', 'Product_ID', 'Date'], ascending=True)
tmp['Sales_Previous_7_days'] = tmp.groupby(by=['Store_ID2', 'Product_ID'], as_index=False)['Quantity'].transform(lambda x: x.rolling(window=7).sum())
tmp = tmp.rename(columns={'Date': 'Next_restock_date'}).drop(columns=['Quantity'])

# # Merge the sls of the next 7 days to the stocks
stocks = stocks.merge(tmp, how='left').rename(columns={'Sales_Previous_7_days': 'Sales_Next_7_days'})

stocks

,Store_ID2,Product_ID,Date,Next_restock_date,Sales_Next_7_days,Sales_Previous_7_days
0,0,0,2016-01-17,2016-01-24,NaN,NaN
1,0,9,2016-01-17,2016-01-24,NaN,NaN
2,0,10,2016-01-17,2016-01-24,NaN,NaN
3,0,44,2016-01-17,2016-01-24,NaN,NaN
4,1,0,2016-01-17,2016-01-24,NaN,NaN
...,...,...,...,...,...,...
750287,3034,533,2022-07-03,2022-07-10,NaN,NaN
750288,3034,2341,2022-07-03,2022-07-10,NaN,NaN
750289,3035,3,2022-07-08,2022-07-15,NaN,NaN
750290,3035,100,2022-07-08,2022-07-15,NaN,NaN


In [54]:
sls.query('Store_ID2 == 2 and Product_ID == 0').to_clipboard()

In [53]:
stocks.query('Store_ID2 == 2 and Product_ID == 0').to_clipboard()

In [34]:

for _, group_df in a:
    continue

91178   NaN
Name: Quantity, dtype: float64

In [18]:
sls.query('Store_ID2 == 0 and Product_ID == 0').to_clipboard()

In [11]:
sls.sort_values(by=['Store_ID2', 'Product_ID', 'Date'], ascending=True)

,Store_ID,Round,Country,Currency,Day,Month,Year,Date,Province,City,...,Dollar_Price,Dollar_Price_Per_Stick_Cigarett,Dollar_Price_Pack_Cigarettes,Fieldworker_Comment,Data_Cleaner_Comment,Product_ID,Fieldworker_ID,Store_ID2,Sales_Next_7_days,Sales_Previous_7_days
0,123,1,Lesotho,LSL,18.0,1.0,2016.0,2016-01-18,Lowlands,Butha-Buthe,...,0.153374,0.153374,3.067485,None,None,0,0,0,NaN,NaN
39,72,1,Lesotho,LSL,18.0,1.0,2016.0,2016-01-18,Lowlands,Butha-Buthe,...,0.153374,0.153374,3.067485,None,None,0,25,0,282.0,NaN
188,76,1,Lesotho,LSL,18.0,1.0,2016.0,2016-01-18,Lowlands,Butha-Buthe,...,0.153374,0.153374,3.067485,None,None,0,66,0,83.0,NaN
211,108,1,Lesotho,LSL,18.0,1.0,2016.0,2016-01-18,Lowlands,Butha-Buthe,...,0.153374,0.153374,3.067485,None,None,0,70,0,102.0,NaN
214,120,1,Lesotho,LSL,18.0,1.0,2016.0,2016-01-18,Lowlands,Butha-Buthe,...,0.153374,0.153374,3.067485,None,None,0,72,0,64.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205353,293,12,South Africa,ZAR,4.0,7.0,2022.0,2022-07-04,KwaZulu-Natal,Durban,...,1.021903,0.102190,2.043806,None,None,533,23747,3034,360.0,NaN
215355,293,12,South Africa,ZAR,4.0,7.0,2022.0,2022-07-04,KwaZulu-Natal,Durban,...,1.498791,0.074940,1.498791,None,None,2341,23747,3034,170.0,NaN
210239,186,12,South Africa,ZAR,9.0,7.0,2022.0,2022-07-09,Western Cape,Cape Town,...,3.746977,0.187349,3.746977,None,None,3,23802,3035,83.0,NaN
211296,186,12,South Africa,ZAR,9.0,7.0,2022.0,2022-07-09,Western Cape,Cape Town,...,3.065708,0.153285,3.065708,None,None,100,23802,3035,282.0,NaN
